In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv')
test = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')
submission = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv')

In [ ]:
test.shape

In [ ]:
submission.shape

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error
from xgboost import plot_importance
from sklearn.model_selection import GridSearchCV,KFold
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor
import gc
import os
import itertools
import sys
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
# matchType : solo, duo, squad 3종류 단일화
'''
solo  <-- solo,solo-fpp,normal-solo,normal-solo-fpp
duo   <-- duo,duo-fpp,normal-duo,normal-duo-fpp,crashfpp,crashtpp
squad <-- squad,squad-fpp,normal-squad,normal-squad-fpp,flarefpp,flaretpp
'''

# 함수선언
mapper = lambda x: 'solo' if ('solo' in x) else 'duo' if ('duo' in x) or ('crash' in x) else 'squad'

# 변경
train['matchType'] = train['matchType'].apply(mapper)

In [ ]:
test.shape

In [ ]:
# 메모리 줄여주는 함수
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                #if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                #    df[col] = df[col].astype(np.float16)
                #el
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        #else:
            #df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB --> {:.2f} MB (Decreased by {:.1f}%)'.format(
        start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# 결측지 제거하고 확인
train.dropna(subset=['winPlacePerc'], inplace = True)
train.isnull().sum()

# 이상치 찾기
# 이상치 제거, 리턴은 DataFrame
def find_outlier(data_col):
  q1, q3 = np.percentile(data_col,[0.01, 99.99])
  iqr = q3 - q1
  lower = q1 - (iqr * 1.5)
  upper = q3 + (iqr * 1.5)

  # 이상치 데이터
  outer_df = train[(data_col > upper) | (data_col < lower)]

  # 이상치를 제외한 데이터
  inner_df = train[(upper >= data_col) & (data_col >= lower)]
  
  return outer_df, inner_df


# 데미지가 0인데 죽이거나 기절시킨 데이터 제거
# 리턴값은 제거된 데이터프레임

def rm_no_damage(data):
  print("이상치  제거 할 DataFrame shape : ", data.shape)
  no_damage = data[(data["damageDealt"] ==0 ) & ((data["kills"] > 0) | (data['DBNOs'] > 0))]

  idx = no_damage.index
  print("이상치 데이터 수 :",len(idx))

  df = data.drop(data.index[idx]).reset_index(drop = True)
  print("이상치 제거 후 DataFrame shape : ", df.shape)
  
  return df



# 1등이 존재하지 않는 매치
def no_winner(data):
  # 의미 -> 매치별로 그룹화해서 'winPlacePerc' max 값 확인 
  # 결과 1이 아니다 -> 해당 매치에 1등이 없다 -> 해당 매치는 오류난 매치
  maxPlacePerc = data.groupby('matchId')['winPlacePerc'].max()
  df = pd.DataFrame(maxPlacePerc[maxPlacePerc != 1]).reset_index()
  print("이상치  제거 할 DataFrame shape : ", data.shape)

  # 해당 'matchId'가 포함된 데이터프레임 확인
  ol = df['matchId']
  ex = data[data['matchId'].isin(ol)]

  # 해당 index 추출
  idx = ex.index
  print("이상치 데이터 수 :",len(idx))

  # 드롭
  df = data.drop(data.index[idx]).reset_index(drop = True)
  print("이상치 제거 후 DataFrame shape : ", df.shape)
  return df


# 'numGroups' 매커니즘은 아마 winPlacePerc가 같은 사람들을 그룹화해서 그 숫자를 센듯
def rm_no_player(data):
  print("이상치  제거 할 DataFrame shape : ", data.shape)

  # 'maxPlace' > 1  : 튕긴 사람을 포함한 플레이어 수는 1명보다 많음 
  # 'numGroups' == 1 : 실질적으로 플레이한 그룹(사람)은 혼자임 -> 혼자 게임
  no_player = data[(data['maxPlace'] > 1) & (data['numGroups'] == 1)]
  
  # 해당 데이터 인덱스추출
  idx = no_player.index
  print("이상치 데이터 수 :",len(idx))

  # 드롭
  df = data.drop(data.index[idx]).reset_index(drop = True)
  print("이상치 제거 후 DataFrame shape : ", df.shape)
  
  return df


def people_num_not_matchtype(data):
  print("이상치  제거 할 DataFrame shape : ", data.shape)

  # 'matchId', 'groupId', 'matchType'을 그룹하여 ID를 카운터해서 'players' 컬럼 추가
  group = data.groupby(['matchId','groupId','matchType'])['Id'].count().to_frame('players').reset_index()

  # 'matchType'과 플레이어 수가 일치하지 않는 경우 확인
  solo_ex = group[(group['matchType'] == 'solo') & (group['players'] != 1)]
  duo_ex = group[(group['matchType'] == 'duo') & (group['players'] > 2)]
  squad_ex = group[(group['matchType'] == 'squad') & (group['players'] > 4)]


  # 매치당 비정상적인 그룹이 포함된  숫자 정보 가진 'Abnormal' 컬럼 생성후 인덱스 초기화
  solo_ex = solo_ex.groupby(['matchId'])['players'].count().to_frame('Abnormal').reset_index()
  duo_ex = duo_ex.groupby(['matchId'])['players'].count().to_frame('Abnormal').reset_index()
  squad_ex = squad_ex.groupby(['matchId'])['players'].count().to_frame('Abnormal').reset_index()


  # 모든 매치에 비이상적인 그룹수를 가진 팀이 존재함
  # 따라서 비이상적인 그룹이 4그룹보다 많으면 매치가 과도하게 오염되었다고 설정
  solo_ex = solo_ex[solo_ex['Abnormal'] > 4]
  duo_ex = duo_ex[duo_ex['Abnormal'] > 4]
  squad_ex = squad_ex[squad_ex['Abnormal'] > 4]

  # 매치 아이디를 리스트로 저장 
  ol_solo = list(solo_ex['matchId'].unique())
  ol_duo = list(duo_ex['matchId'].unique())
  ol_squad = list(squad_ex['matchId'].unique())
  print("비이상적인 그룹이 4그룹보다 많은 솔로 매치 타입 수", len(ol_solo))
  print("비이상적인 그룹이 4그룹보다 많은 듀오 매치 타입 수", len(ol_duo))
  print("비이상적인 그룹이 4그룹보다 많은 스쿼드 매치 타입 수",len(ol_squad))


  # 각 매치별 오염된 매치 통합후 제거
  # 이제까지 구한 모든 리스트를 합쳐서 통합한다.
  ol_total = ol_solo + ol_duo + ol_squad
  print("모든 매치에 비이상적인 그룹이 4그룹보다 많은 경우 수",len(ol_total))

  # 비정상적인 'matchId'가 포함된 데이터프레임
  ex = data[data['matchId'].isin(ol_total)]
  
  # 인덱스 저장
  idx = ex.index
  print("제거할 이상치 수",len(idx))

  # 인덱스로 드롭
  df = data.drop(data.index[idx]).reset_index(drop = True)
  print("이상치 제거 후 DataFrame shape : ", df.shape)
  return df


def odd_mathch_kill(data):

  print("이상치  제거 할 DataFrame shape : ", data.shape)
  grp = train8.groupby('matchId',as_index = False).sum()

  # 매치별 킬 수 합이 99가 넘는 데이터들을 뽑는다.
  ol = grp[grp['kills'] > 99]['matchId']

  # train에서 ol 에 존재하는 매치에 해당하는 데이터들을 모두 본다.
  ex = data[data['matchId'].isin(ol)]

  # 인덱스 추출
  idx = ex.index
  print("이상치 데이터 수 :",len(idx))

  # 이상치 제거후 인덱스 재정렬
  df = data.drop(data.index[idx]).reset_index(drop = True)
  print("이상치 제거 후 DataFrame shape : ", df.shape)
  
  return df


def overkill_people_match(data):

  # 매치 내
  ex = data[data['kills'] >= data['maxPlace']]['matchId'].unique()

  ol = list(ex)

  # isin함수를 사용 
  # train에서 ol에 존재하는 매치에 해당하는 데이터들을 모두 본다.
  ex = data[data['matchId'].isin(ol)]


  # 인덱스 정보를 idx 변수에 저장한다.
  idx = ex.index

  # ex에 해당하는 인덱스와 일치하는 인덱스를 가진 데이터를 전부 날린다
  # 원본을 건드리지 말고 임시로 temp에 넣는다.
  df = data.drop(data.index[idx]).reset_index(drop = True)

  return df


# 운전별로안하고 로드킬 비정상적으로 많이한 놈 잡기

# 운전거리 최소 1미터에 1킬한다고 가정
def roadkill_bug(data):

  print("이상치  제거 할 DataFrame shape : ", data.shape)
  out_roadkill = find_outlier(train['roadKills'])[0]

  roadkill_num = out_roadkill['roadKills'].unique()
  print("로드킬 상위 0.01% 킬 수", roadkill_num)

  road_bug = data[(data['rideDistance']* 100 < roadkill_num.max()) &  (data['roadKills'] > roadkill_num.min())]
  idx = road_bug.index
  print("이상치 데이터 수 :",len(idx))

  df = data.drop(data.index[idx]).reset_index(drop = True)
  print("이상치 제거 후 DataFrame shape : ", df.shape)

  return df



def no_weapon_solo(data):
  
  print("이상치  제거 할 DataFrame shape : ", data.shape)
  
  # 1등인데, 무기 획득 = 0 이고 매치 타입이 'solo'인 경우 
  no_weapon_solo_df = data[((train['winPlacePerc'] > 0.99) & (data['weaponsAcquired'] == 0) & (data['matchType'] == "solo"))]
  
  # 위 경우의 인덱스 추출
  idx = no_weapon_solo_df.index
  print("이상치 데이터 수 :",len(idx))

  # 드롭
  df = data.drop(data.index[idx]).reset_index(drop = True)
  
  print("이상치 제거 후 DataFrame shape : ", df.shape)
  return df

In [ ]:
train3 = reduce_mem_usage(rm_no_damage(train))

train4 = reduce_mem_usage(no_winner(train3))

train5 =  reduce_mem_usage(rm_no_player(train4))

train8 = reduce_mem_usage(people_num_not_matchtype(train5))

train9 = reduce_mem_usage(odd_mathch_kill(train8))

train10 = reduce_mem_usage(overkill_people_match(train9))

train11 = reduce_mem_usage(roadkill_bug(train10))

train12 = reduce_mem_usage(no_weapon_solo(train11))

In [ ]:
all_data  = train12.copy()


In [ ]:
test.shape

# train

In [ ]:
all_data.head()

In [ ]:
all_data = all_data.sort_values(['matchId','groupId','Id','kills']).reset_index().drop('index',axis = 1)

# 매치별로 그룹화해서 match라는 변수로 만듦
match = all_data.groupby('matchId')

# 비율스텟을 만들어준다 
# 킬수가 높을수록 1에 가까운 것이다.
all_data['killsPerc'] = match['kills'].rank(pct=True).values
all_data['killPlacePerc'] = match['killPlace'].rank(pct=True).values
all_data['walkDistancePerc'] = match['walkDistance'].rank(pct=True).values
# all_data['damageDealtPerc'] = match['damageDealt'].rank(pct=True).values
all_data['walkPerc_killsPerc'] = all_data['walkDistancePerc'] / all_data['killsPerc']
# 전체 이동거리 
all_data['_totalDistance'] = all_data['rideDistance'] + all_data['walkDistance'] + all_data['swimDistance']
# 힐템 전체 사용량
all_data['_healthItems'] = all_data['heals'] + all_data['boosts']
# 헤드샷 킬의 비율
all_data['_headshotKillRate'] = all_data['headshotKills'] / all_data['kills']
# 킬순위 / 매치인원 수 즉 킬순위의 상대적 위치 비교하는 칼럼 
all_data['_killPlaceOverMaxPlace'] = all_data['killPlace'] / all_data['maxPlace']
# 걸어다닌 거리당 킬 수
all_data['_killsOverWalkDistance'] = all_data['kills'] / all_data['walkDistance']


# 비율 스탯을 계산하다 생길 수 있는 Inf를 val값으로 채우는 함수 생성
def fillInf(df, val):
    numcols = df.select_dtypes(include='number').columns
    cols = numcols[numcols != 'winPlacePerc']
    df[df == np.Inf] = np.NaN
    df[df == np.NINF] = np.NaN
    for c in cols: df[c].fillna(val, inplace=True)

# 무한대가 나온 항목을 0으로 채운다 
fillInf(all_data, 0)

# 필요없다고 생각하는 특징 지우기 
all_data.drop(['boosts','heals','killStreaks','DBNOs'], axis=1, inplace=True)
all_data.drop(['headshotKills','roadKills','vehicleDestroys'], axis=1, inplace=True)
all_data.drop(['rideDistance','swimDistance','matchDuration'], axis=1, inplace=True)
all_data.drop(['rankPoints','killPoints','winPoints'], axis=1, inplace=True)


# 필요한 단위별로 묶어서 변수에 넣어둔다.
match = all_data.groupby(['matchId'])
group = all_data.groupby(['matchId','groupId','matchType'])

# 필요한 칼럼이름만 남겨서 변수에 리스트화 해 둔다.
agg_col = list(all_data.columns)
exclude_agg_col = ['Id','matchId','groupId','matchType','maxPlace','numGroups','winPlacePerc']
for c in exclude_agg_col:
    agg_col.remove(c)
print(agg_col)

# 주요(합을 시행할 )칼럼을 변수에 넣어둔다.
sum_col = ['kills','killPlace','damageDealt','walkDistance','_healthItems']

# 이를 합쳐서 matchdata를 생성한다.
# 활용에 용이성을 위해 인덱스는 풀어준다.
match_data = pd.concat([
    match.size().to_frame('m.players'), 
    match[sum_col].sum().rename(columns=lambda s: 'm.sum.' + s), 
    match[sum_col].max().rename(columns=lambda s: 'm.max.' + s),
    match[sum_col].mean().rename(columns=lambda s: 'm.mean.' + s)
    ], axis=1).reset_index()


# 매치데이터에 추가해준다
match_data = pd.merge(match_data, 
    group[sum_col].sum().rename(columns=lambda s: 'g_sum.' + s).reset_index())


# 그룹별 통계
# agg_col은 위에서 만든 리스트로 
# ['assists', 'damageDealt', 'killPlace', 'kills', 'longestKill', 'revives', 'teamKills', 'walkDistance', 'weaponsAcquired', 'killsPerc', 'killPlacePerc', 'walkDistancePerc', 'walkPerc_killsPerc', '_totalDistance', '_healthItems', '_headshotKillRate', '_killPlaceOverMaxPlace', '_killsOverWalkDistance']
# 이다. 너무 길당 
# 해당 그룹단위로 사람수, 평균, agg_col에 해당하는 요소의 max와 min값 
all_data = pd.concat([
    group.size().to_frame('players'),
    group.mean(),
    group[agg_col].max().rename(columns=lambda s: 'g_max.' + s),
    group[agg_col].min().rename(columns=lambda s: 'g_min.' + s),
    ], axis=1).reset_index()
# 메모리 절약
all_data = reduce_mem_usage(all_data)

print(all_data.shape)


# 힘들게 만든 매치별 데이터를 본래 데이터와 합쳐준다.
all_data = pd.merge(all_data, match_data)

# 메모리 절약용
del match_data
gc.collect()

# 매치타입의 종류에 따른 열을 만들고
# 즉 solo , duo , squad 가 생기고
# 매치타입이 일치하면 1 아니면 0을 넣는다.
pd.concat([all_data, pd.get_dummies(all_data['matchType'])], axis=1)

# 적용
all_data = pd.concat([all_data, pd.get_dummies(all_data['matchType'])], axis=1)
# 매치 타입은 이제 버림
all_data.drop(['matchType'], axis=1, inplace=True)

# test

In [ ]:
all_data1 =  test.copy()
mapper = lambda x: 'solo' if ('solo' in x) else 'duo' if ('duo' in x) or ('crash' in x) else 'squad'
all_data1['matchType'] = all_data1['matchType'].apply(mapper)

In [ ]:
all_data1 = all_data1.sort_values(['matchId','groupId','Id','kills']).reset_index().drop('index',axis = 1)

# 매치별로 그룹화해서 match라는 변수로 만듦
match = all_data1.groupby('matchId')

# 비율스텟을 만들어준다 
# 킬수가 높을수록 1에 가까운 것이다.
all_data1['killsPerc'] = match['kills'].rank(pct=True).values
all_data1['killPlacePerc'] = match['killPlace'].rank(pct=True).values
all_data1['walkDistancePerc'] = match['walkDistance'].rank(pct=True).values
# all_data['damageDealtPerc'] = match['damageDealt'].rank(pct=True).values
all_data1['walkPerc_killsPerc'] = all_data1['walkDistancePerc'] / all_data1['killsPerc']
# 전체 이동거리 
all_data1['_totalDistance'] = all_data1['rideDistance'] + all_data1['walkDistance'] + all_data1['swimDistance']
# 힐템 전체 사용량
all_data1['_healthItems'] = all_data1['heals'] + all_data1['boosts']
# 헤드샷 킬의 비율
all_data1['_headshotKillRate'] = all_data1['headshotKills'] / all_data1['kills']
# 킬순위 / 매치인원 수 즉 킬순위의 상대적 위치 비교하는 칼럼 
all_data1['_killPlaceOverMaxPlace'] = all_data1['killPlace'] / all_data1['maxPlace']
# 걸어다닌 거리당 킬 수
all_data1['_killsOverWalkDistance'] = all_data1['kills'] / all_data1['walkDistance']


# 비율 스탯을 계산하다 생길 수 있는 Inf를 val값으로 채우는 함수 생성
def fillInf(df, val):
    numcols = df.select_dtypes(include='number').columns
    cols = numcols[numcols != 'winPlacePerc']
    df[df == np.Inf] = np.NaN
    df[df == np.NINF] = np.NaN
    for c in cols: df[c].fillna(val, inplace=True)

# 무한대가 나온 항목을 0으로 채운다 
fillInf(all_data, 0)

# 필요없다고 생각하는 특징 지우기 
all_data1.drop(['boosts','heals','killStreaks','DBNOs'], axis=1, inplace=True)
all_data1.drop(['headshotKills','roadKills','vehicleDestroys'], axis=1, inplace=True)
all_data1.drop(['rideDistance','swimDistance','matchDuration'], axis=1, inplace=True)
all_data1.drop(['rankPoints','killPoints','winPoints'], axis=1, inplace=True)


# 필요한 단위별로 묶어서 변수에 넣어둔다.
match = all_data1.groupby(['matchId'])
group = all_data1.groupby(['matchId','groupId','matchType'])

# 필요한 칼럼이름만 남겨서 변수에 리스트화 해 둔다.
agg_col = list(all_data1.columns)
exclude_agg_col = ['Id','matchId','groupId','matchType','maxPlace','numGroups']
for c in exclude_agg_col:
    agg_col.remove(c)
print(agg_col)

# 주요(합을 시행할 )칼럼을 변수에 넣어둔다.
sum_col = ['kills','killPlace','damageDealt','walkDistance','_healthItems']

# 이를 합쳐서 matchdata를 생성한다.
# 활용에 용이성을 위해 인덱스는 풀어준다.
match_data = pd.concat([
    match.size().to_frame('m.players'), 
    match[sum_col].sum().rename(columns=lambda s: 'm.sum.' + s), 
    match[sum_col].max().rename(columns=lambda s: 'm.max.' + s),
    match[sum_col].mean().rename(columns=lambda s: 'm.mean.' + s)
    ], axis=1).reset_index()


# 매치데이터에 추가해준다
match_data = pd.merge(match_data, 
    group[sum_col].sum().rename(columns=lambda s: 'g_sum.' + s).reset_index())


# 그룹별 통계
# agg_col은 위에서 만든 리스트로 
# ['assists', 'damageDealt', 'killPlace', 'kills', 'longestKill', 'revives', 'teamKills', 'walkDistance', 'weaponsAcquired', 'killsPerc', 'killPlacePerc', 'walkDistancePerc', 'walkPerc_killsPerc', '_totalDistance', '_healthItems', '_headshotKillRate', '_killPlaceOverMaxPlace', '_killsOverWalkDistance']
# 이다. 너무 길당 
# 해당 그룹단위로 사람수, 평균, agg_col에 해당하는 요소의 max와 min값 
all_data1 = pd.concat([
    group.size().to_frame('players'),
    group.mean(),
    group[agg_col].max().rename(columns=lambda s: 'g_max.' + s),
    group[agg_col].min().rename(columns=lambda s: 'g_min.' + s),
    ], axis=1).reset_index()
# 메모리 절약
all_data1 = reduce_mem_usage(all_data1)

print(all_data1.shape)


# 힘들게 만든 매치별 데이터를 본래 데이터와 합쳐준다.
all_data1 = pd.merge(all_data1, match_data)

# 메모리 절약용
del match_data
gc.collect()

# 매치타입의 종류에 따른 열을 만들고
# 즉 solo , duo , squad 가 생기고
# 매치타입이 일치하면 1 아니면 0을 넣는다.
pd.concat([all_data1, pd.get_dummies(all_data1['matchType'])], axis=1)

# 적용
all_data1 = pd.concat([all_data1, pd.get_dummies(all_data1['matchType'])], axis=1)
# 매치 타입은 이제 버림
all_data1.drop(['matchType'], axis=1, inplace=True)

In [ ]:
test = all_data1.copy()

In [ ]:
tempID = test[['matchId','groupId']]
tempID

In [ ]:
# ID를 날려준다
test = test.drop(['matchId','groupId'],axis = 1)

In [ ]:
test.head()

# test

In [ ]:
train = all_data.copy()

In [ ]:
X = train.drop(['winPlacePerc','matchId','groupId'],axis = 1)
y = train['winPlacePerc']

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# LGbm -r

In [ ]:
model_lgb = LGBMRegressor(
    n_estimators = 3000,
    max_depth = 8,
    min_child_weight = 8,
    learning_rate = 0.1
)
model_lgb.fit(X_train, y_train)

pred = model_lgb.predict(X_test)
print("MSE : {:.6f}".format(mean_squared_error(pred, y_test)))
print("MAE : {:.6f}".format(mean_absolute_error(pred, y_test)))
print("R2_Score : {:.6f}".format(r2_score(pred, y_test)))
print("RMSE : {:.6f}".format(np.sqrt(mean_squared_error(pred, y_test))))

In [ ]:
pred1 = model_lgb.predict(test)

In [ ]:
test['winPlacePerc'] = pred1

In [ ]:
# 매치랑 그룹ID 원상복구
test = pd.concat([tempID,test],axis=1)
test

In [ ]:
all_dd = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
# 원본 테스트에서 ID 정보만 뽑아와
ID = all_dd[['Id','groupId','matchId']]

In [ ]:
# 머지를 통해 병합
final = pd.merge(ID,test)

In [ ]:
# 결과 확인 
final

In [ ]:
final_1 = final.copy()

In [ ]:
submission.drop('winPlacePerc',axis = 1,inplace = True)

In [ ]:
submission

In [ ]:
submission = pd.merge(submission,final)

In [ ]:
submission  = submission[['Id', 'winPlacePerc']]

In [ ]:
submission

In [ ]:
submission.to_csv('./submission.csv', index= False)

# xgb

In [ ]:
# model_xgb = XGBRegressor(
#     n_estimators = 150,
#     max_depth = 5,
#     min_child_weight = 7,
#     learning_rate = 0.1
# )

# model_xgb.fit(X_train,y_train)

In [ ]:
# predictions = model_xgb.predict(X_test)

In [ ]:
# predictions

In [ ]:
# y_test.values


In [ ]:
# print("MSE :",mean_squared_error(predictions, y_test))
# print("MAE :",mean_absolute_error(predictions, y_test))

In [ ]:
# plot_importance(model_xgb)

In [ ]:
# fig = plt.figure( figsize = (20, 20) )
# chart = fig.add_subplot(1,1,1)
# chart.plot(predictions[:100], marker='o', color='blue', label='predict value')
# chart.plot(y_test.values[:100], marker='^', color='red', label='real value')
# chart.set_title('real value vs predict value')
# plt.xlabel('index')
# plt.ylabel('real vs predict')
# plt.legend(loc = 'best') 

In [ ]:
#   xgb.plot_tree(my_model, num_trees=0, rankdir='LR')

#   fig = plt.gcf()
#   fig.set_size_inches(150, 100)

#   plt.show()

# lgbm


In [ ]:
# model_lgb = LGBMRegressor()
# model_lgb.fit(X_train, y_train)


In [ ]:
# predictions = model_lgb.predict(X_test)

In [ ]:
# print("MSE :",mean_squared_error(predictions, y_test))
# print("MAE :",mean_absolute_error(predictions, y_test))

In [ ]:
# model_lgb = LGBMRegressor(
#     n_estimators = 20000,
#     max_depth = 8,
#     min_child_weight = 8,
#     colsample_bytree = 0.7,
#     learning_rate = 0.05
# )


# model_lgb.fit(X_train, y_train)

In [ ]:
# predictions = model_lgb.predict(X_test)
# print("MSE :",mean_squared_error(predictions, y_test))
# print("MAE :",mean_absolute_error(predictions, y_test))